In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from torch import nn,optim
import torch

import algo

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
import numpy as np
import pandas as pd

import attack
import random




2023-02-10 15:08:12.480907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

class Train_args():

    def __init__(self, learning_rate, weight_decay, epoch):
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.epoch = epoch
         
def train_attack_model(model, train_data, train_target, train_args):

    optimizer = optim.Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)
    
    model.train()
    optimizer.zero_grad()
    output = model(train_data)
    loss = nn.CrossEntropyLoss()(output, train_target.to(torch.long))
    loss.backward(retain_graph=True)
    optimizer.step()

    return model

def attack_evaluation(model, x, y, dev="cpu", extended=False):

    model.eval()

    with torch.no_grad():
        output =  model(x)
        out_target = output.argmax(1, keepdim=True)
        correct = out_target.to(dev).eq(y.to(dev).view_as(out_target.to(dev))).sum().item()
        acc = correct/y.shape[0]

        predicted_positive = output.argmax(1, keepdim=True) == 1
        labeled_positive = y == 1
        tp = predicted_positive.to(dev) * labeled_positive.to(dev).view_as(out_target)
        tp_count = tp.to(dev).sum().item()
        
        if predicted_positive.to(dev).sum().item() != 0:
            pre = tp_count / predicted_positive.to(dev).sum().item()
        else:
            pre = 0
        if labeled_positive.to(dev).sum().item() !=0:
            rec = tp_count / labeled_positive.to(dev).sum().item()
        else:
            rec = 0
    if extended:
        predicted_negative = output.argmax(1, keepdim=True) == 0
        labeled_negative = y == 0
        tn = predicted_negative.to(dev) * labeled_negative.to(dev).view_as(out_target)
        tn_count = tn.to(dev).sum().item()

        fp_count = predicted_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        fn_count = labeled_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        
        return acc, pre, rec, tp_count, tn_count, fp_count, fn_count
    else:
        return acc, pre, rec


In [7]:
# raw_data = pd.read_csv('../datasets/dataset_purchase', )
# y=raw_data['63']
# X=raw_data.drop('63', axis=1)
# y =  y.replace(100, 0)
# print(y.nunique())

# X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.1, random_state=42)
# print(X_train.shape, x_shadow.shape)

100
(19732, 600) (177591, 600)


In [3]:
data = pd.read_csv('../DP-UTIL.nosync/loan_preprocessed.csv', index_col=0)


In [4]:
data.shape

(661283, 167)

In [24]:
#Loan
y = data['grade']
X = data.drop('grade', axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
tr_size = 10000
X_train, X_test, y_train, y_test = train_test_split(X[0:100000], y[0:100000], train_size=0.5, random_state=0)
x_target_train = np.array(X_train)[:tr_size]
y_target_train = np.array(y_train)[:tr_size]
x_target_test = np.array(X_test)[:tr_size]
y_target_test = np.array(y_test)[:tr_size]

x_shadow = X[100000:]
y_shadow = y[100000:]

In [25]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.001
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False
model.L              = 1
model.epsilon        = 0

In [26]:
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

The accuracy of the model : 88.0 %
The accuracy of the model : 83.7 %


0.8371

In [12]:
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)

The accuracy of the model : 84.39999999999999 %
The accuracy of the model : 57.99999999999999 %


0.58

In [14]:
s_ms = {}
number_of_sms = 10
shadow_size = 10000
shadow_batch_size = int(shadow_size/number_of_sms)

x_shadow_train = np.array(x_shadow[:shadow_size])
y_shadow_train = np.array(y_shadow[:shadow_size])
x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

for i in range(number_of_sms):
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    shadow_model = algo.LogisticRegression_DPSGD()

    shadow_model.n_classes      = len(np.unique(y_target_test))
    shadow_model.alpha          = 0.001
    shadow_model.max_iter       = 100
    shadow_model.lambda_        = 10e-5
    shadow_model.tolerance      = 10e-5
    shadow_model.DP             = False

    X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
    shadow_model.SGD(X,y)
    print('Shadow model: ', i)
    shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end], acc=True)
    shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end], acc=True)
    s_ms[i] = shadow_model

/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_47583/701540644.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_train = np.array(y_shadow[:shadow_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_47583/701540644.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])


Shadow model:  0
The accuracy of the model : 84.1 %
The accuracy of the model : 59.4 %
Shadow model:  1
The accuracy of the model : 84.8 %
The accuracy of the model : 60.5 %
Shadow model:  2
The accuracy of the model : 85.7 %
The accuracy of the model : 57.4 %
Shadow model:  3
The accuracy of the model : 81.0 %
The accuracy of the model : 62.3 %
Shadow model:  4
The accuracy of the model : 86.5 %
The accuracy of the model : 56.00000000000001 %
Shadow model:  5
The accuracy of the model : 83.7 %
The accuracy of the model : 61.4 %
Shadow model:  6
The accuracy of the model : 85.0 %
The accuracy of the model : 56.39999999999999 %
Shadow model:  7
The accuracy of the model : 85.3 %
The accuracy of the model : 61.9 %
Shadow model:  8
The accuracy of the model : 83.2 %
The accuracy of the model : 57.599999999999994 %
Shadow model:  9
The accuracy of the model : 84.2 %
The accuracy of the model : 55.60000000000001 %


In [15]:
shadow_train_pred = []
shadow_test_pred = []

for i in range(number_of_sms): 
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
    test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
    shadow_train_pred.append(train_prediciton)
    shadow_test_pred.append(test_prediciton)
    
    

In [16]:
y_shadow_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_train.reshape(-1,1)) #encoode the target values
y_shadow_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_test.reshape(-1,1)) #encoode the target values


In [17]:
shadow_train_pred[0].shape

(1000, 7)

In [18]:
sh_train_pred = np.concatenate(shadow_train_pred)
sh_test_pred = np.concatenate(shadow_test_pred)

# members
labels = np.ones(sh_train_pred.shape[0])
# non-members
test_labels = np.zeros(sh_test_pred.shape[0])

x_1 = np.concatenate((sh_train_pred, sh_test_pred))
x_2 = np.concatenate((y_shadow_train_ohe, y_shadow_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))


In [19]:
attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)

In [20]:
# attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
# attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-5, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
    attack_model = attack.train_attack_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    print(f'epoch {epoch}: {attack_evaluation(attack_model, attack_train_data, attack_train_target)}' )        


epoch 0: (0.5168, 0.5436590436590436, 0.2092)
epoch 1: (0.5153, 0.5384422110552763, 0.2143)
epoch 2: (0.51035, 0.5248977628097186, 0.2182)
epoch 3: (0.50225, 0.5050732807215332, 0.224)
epoch 4: (0.49565, 0.4910585817060637, 0.2389)
epoch 5: (0.5034, 0.5048158640226629, 0.3564)
epoch 6: (0.52905, 0.526639156350298, 0.5743)
epoch 7: (0.5241, 0.5193449991973029, 0.647)
epoch 8: (0.51985, 0.5150755677071467, 0.6782)
epoch 9: (0.5226, 0.517168034032209, 0.6808)
epoch 10: (0.52385, 0.5183815028901734, 0.6726)
epoch 11: (0.5248, 0.5193146417445483, 0.6668)
epoch 12: (0.5319, 0.5243846506650359, 0.686)
epoch 13: (0.5418, 0.5304044224614489, 0.7292)
epoch 14: (0.55215, 0.5366453516969996, 0.7637)
epoch 15: (0.5606, 0.541558085310657, 0.7897)
epoch 16: (0.5696, 0.5469065911847958, 0.8115)
epoch 17: (0.5743, 0.5493687707641196, 0.8268)
epoch 18: (0.57885, 0.5524373212741903, 0.8307)
epoch 19: (0.5813, 0.5547179970386323, 0.8242)
epoch 20: (0.58375, 0.5564924114671164, 0.825)
epoch 21: (0.5869, 0.

epoch 175: (0.6315, 0.609674728940784, 0.731)
epoch 176: (0.6312, 0.611678583588696, 0.7186)
epoch 177: (0.63205, 0.6107894957630674, 0.728)
epoch 178: (0.6314, 0.6119631901840491, 0.7182)
epoch 179: (0.6318, 0.6098150308281953, 0.7319)
epoch 180: (0.6311, 0.6115175229669956, 0.7189)
epoch 181: (0.63185, 0.6099391311598432, 0.7315)
epoch 182: (0.63145, 0.6117297067573311, 0.7197)
epoch 183: (0.63175, 0.6096911164765632, 0.7323)
epoch 184: (0.63105, 0.6112762163539102, 0.7199)
epoch 185: (0.63195, 0.609784507862551, 0.7329)
epoch 186: (0.63115, 0.6113233172056701, 0.7202)
epoch 187: (0.63185, 0.6096830546543549, 0.7329)
epoch 188: (0.63105, 0.6110875646350767, 0.7209)
epoch 189: (0.63205, 0.609648758614963, 0.7342)
epoch 190: (0.631, 0.6110734271663558, 0.7207)
epoch 191: (0.6322, 0.6098554096725943, 0.7339)
epoch 192: (0.63085, 0.610861645344404, 0.721)
epoch 193: (0.63195, 0.6094929881337648, 0.7345)
epoch 194: (0.63105, 0.6109371031913993, 0.7217)
epoch 195: (0.6319, 0.60933355437665

epoch 351: (0.63685, 0.6083788706739527, 0.7682)
epoch 352: (0.636, 0.610032362459547, 0.754)
epoch 353: (0.6371, 0.6085338822039266, 0.7687)
epoch 354: (0.63595, 0.6100542378369627, 0.7536)
epoch 355: (0.63705, 0.6085544554455445, 0.7683)
epoch 356: (0.6363, 0.6101859337105902, 0.7548)
epoch 357: (0.6372, 0.6085614812470328, 0.7691)
epoch 358: (0.6366, 0.6104999191069406, 0.7547)
epoch 359: (0.6372, 0.6085786641342197, 0.769)
epoch 360: (0.6367, 0.610491432266408, 0.7553)
epoch 361: (0.63705, 0.6083999050858182, 0.7692)
epoch 362: (0.6367, 0.6105271668822768, 0.7551)
epoch 363: (0.637, 0.6083346512731298, 0.7693)
epoch 364: (0.6365, 0.6102228682170543, 0.7557)
epoch 365: (0.63705, 0.6082628959633463, 0.77)
epoch 366: (0.6368, 0.6105008077544426, 0.7558)
epoch 367: (0.6371, 0.6083109495970928, 0.77)
epoch 368: (0.63665, 0.6102816560406746, 0.7562)
epoch 369: (0.63715, 0.608273466487724, 0.7705)
epoch 370: (0.6368, 0.6104294478527608, 0.7562)
epoch 371: (0.63705, 0.6081945211968106, 0.7

In [21]:
train_res = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [27]:
test_res = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)

In [28]:
print(train_res)
test_res


(0.6377, 0.6076454033771107, 0.7773)


(0.5247, 0.5145208700764257, 0.8752)

In [61]:
sh_path = f'mia/shms{number_of_sms}_shtrsize{shadow_size}_shlr{shadow_model.alpha}_shiter{int(shadow_model.max_iter/shadow_batch_size)}_shreg{shadow_model.lambda_}/'      
os.mkdir(sh_path)

torch.save(attack_train_data, sh_path+'attack_train_data.pt')
torch.save(attack_train_target, sh_path+'attack_train_target.pt')

at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
torch.save(attack_model, at_path)


In [74]:
train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
train_acc, train_pre, train_rec 

(0.59217, 0.5930014328093153, 0.5877)

In [75]:
test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

(0.6260389215487533, 0.5935737236701178, 0.8313)

In [ ]:
# MI optimization

In [ ]:
ams

In [87]:
rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

epo = [200]
wd = [1e-5, 1e-6]
ams = {}
for ep in epo:
    for w_d in wd:
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=ep)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        for epoch in range(attack_train_args.epoch):

            attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
            ams[(l_r,w_d)] = attack_model
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        torch.save(attack_model, at_path)


In [91]:
epo = [200,500]
wd = [1e-5, 1e-6]

for ep in epo:
    for w_d in wd:
        print(ep, w_d)
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=500)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        attack_model = torch.load(at_path)
        
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print('Train: ', train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print('Test: ' , test_acc, test_pre, test_rec)


200 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
200 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99
500 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
500 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99


In [ ]:
for l_r in lr:
    for w_d in wd:
        ams[(l_r,w_d)]
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print("Train acc pre rec: ", train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print("Test acc pre rec: ", test_acc, test_pre, test_rec)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
shadow_size = 20000
shadow_clf = LogisticRegression(random_state=1).fit(x_shadow[:shadow_size], y_shadow[:shadow_size])

In [ ]:
print(shadow_clf.score(x_shadow[:shadow_size], y_shadow[:shadow_size]))
print(shadow_clf.score(x_shadow[shadow_size:shadow_size*2], y_shadow[shadow_size:shadow_size*2]))

In [ ]:
shadow_train_pred = shadow_clf.predict_proba(x_shadow[:shadow_size])
shadow_test_pred = shadow_clf.predict_proba(x_shadow[shadow_size:shadow_size*2])
y_shadow_train = y_shadow[:shadow_size]
y_shadow_test = y_shadow[shadow_size:shadow_size*2]

In [ ]:
# members
labels = np.ones(shadow_train_pred.shape[0])
# non-members
test_labels = np.zeros(shadow_test_pred.shape[0])

x_1 = np.concatenate((shadow_train_pred, shadow_test_pred))
x_2 = np.concatenate((y_shadow_train, y_shadow_test)).reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = np.array(df.drop(['a_target'], axis=1))
attack_train_target = np.array(df['a_target'])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=200)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
            
    attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [ ]:
train_acc, train_pre, train_rec 

In [67]:
train_pred = model.predict(x_target_train, y_target_train)
test_pred = model.predict(x_target_test, y_target_test)

y_target_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_train.reshape(-1,1)) #encoode the target values
y_target_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_test.reshape(-1,1)) #encoode the target values
    
# members
labels = np.ones(train_pred.shape[0])
# non-members
test_labels = np.zeros(test_pred.shape[0])

x_1 = np.concatenate((train_pred, test_pred))
x_2 = np.concatenate((y_target_train_ohe, y_target_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_test_data = np.concatenate((x_1,x_2),axis=1)
attack_test_target = y_new
df = pd.DataFrame(attack_test_data)
df['a_target'] = attack_test_target
df = df.sample(frac = 1)

attack_test_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_test_target = torch.tensor(np.array(df['a_target']))


In [ ]:
test_acc, test_pre, test_rec = attack_evaluation(a_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

In [14]:
a_model = Net_attack(h_neurons=64, do=0, input_size=100)
a_model = torch.load('attack_model') 

In [ ]:

rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

lr = [0.001]
wd = [1e-4, 1e-6]
tms = {}
for l_r in lr:
    for w_d in wd:
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = 100
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size 
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.epsilon

        X,y = model.init_theta(x_target_train, y_target_train)
        model.SGD(X,y)
        model.evaluate(x_target_train, y_target_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import algo
# import attack

# from torch import nn,optim
import torch

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
        

raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X=raw_data.drop('63', axis=1)
y =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")

for L in [1,10,100]:
    for epsilon in np.arange(0,1,0.1):
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = n_classes
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.L              = L
        model.epsilon        = epsilon


        X,y = model.init_theta(x_target_train, y_target_train)
        model.train(X,y)
        model.evaluate(x_target_train, y_target_train, acc=True)
        model.evaluate(x_target_test, y_target_test, acc=True)

        tm_path = f'tm/lr{model.alpha}_iter{int(model.max_iter/X_train_size)}_reg{model.lambda_}_DP{model.DP}'
        if model.DP:
            tm_path += f'_eps{model.epsilon}_L{model.L}'
        np.save(tm_path+'_target_model', model.theta)

        print(tm_path)

    

In [ ]:
#Shadow models
# s_ms = {}
# number_of_sms = 10
# shadow_size = 50000
# shadow_batch_size = int(shadow_size/number_of_sms)

# x_shadow_train = np.array(x_shadow[:shadow_size])
# y_shadow_train = np.array(y_shadow[:shadow_size])
# x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
# y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

# attack.train_shadow_models(number_of_sms,)

# for i in range(number_of_sms):  
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     shadow_model = algo.LogisticRegression_DPSGD()

#     shadow_model.n_classes      = n_classes
#     shadow_model.alpha          = 0.001
#     shadow_model.max_iter       = 100*shadow_batch_size
#     shadow_model.lambda_        = 10e-3
#     shadow_model.tolerance      = 10e-5
#     shadow_model.DP             = False

#     X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
#     shadow_model.SGD(X,y)
#     print('Shadow model: ', i)
#     shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
#     s_ms[i] = shadow_model

# #Attack model

# shadow_train_pred = []
# shadow_test_pred = []

# for i in range(number_of_sms): 
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
#     shadow_train_pred.append(train_prediciton)
#     shadow_test_pred.append(test_prediciton)
    

In [10]:
1/10000 == 1e-5


False

In [3]:
np.arange(0,1,0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])